# Contextual question answering

In [1]:
!pip install peft huggingface_hub

In [2]:
!pip install evaluate

In [1]:
import json
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import evaluate
import torch
from peft import get_peft_model, LoraConfig, TaskType
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Load poquad data

In [3]:
with open("poquad-train.json", 'r', encoding="utf8") as file:
    train_data = json.load(file)['data']
with open("poquad-dev.json", 'r', encoding="utf8") as file:
    val_data = json.load(file)['data']

In [4]:
def prepare_data(data):
    processed_data = []
    for entry in data:
        for paragraph in entry['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = ""
                if 'answers' in qa and qa['answers']:
                    answer_obj = qa['answers'][0]
                    answer = answer_obj.get('generative_answer', answer_obj.get('text', ""))
                elif 'plausible_answers' in qa and qa['plausible_answers']:
                    answer_obj = qa['plausible_answers'][0]
                    answer = answer_obj.get('generative_answer', answer_obj.get('text', ""))
                processed_data.append({
                    "input": f"context: {context} question: {question}",
                    "output": answer
                })
    return processed_data

In [5]:
train_dataset = prepare_data(train_data)
val_dataset = prepare_data(val_data)

with open("prepared_train.json", "w", encoding='utf-8') as file:
    json.dump(train_dataset, file, indent=4, ensure_ascii=False)

with open("prepared_val.json", "w", encoding='utf-8') as file:
    json.dump(val_dataset, file, indent=4,  ensure_ascii=False)


In [6]:
train_data = load_dataset('json', data_files='prepared_train.json', split='train')
val_data = load_dataset('json', data_files='prepared_val.json', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(train_data[0])

{'input': 'context: Projekty konfederacji zaczęły się załamywać 5 sierpnia 1942. Ponownie wróciła kwestia monachijska, co uaktywniło się wymianą listów Ripka – Stroński. Natomiast 17 sierpnia 1942 doszło do spotkania E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Polscy dyplomaci zaproponowali podpisanie układu konfederacyjnego. W następnym miesiącu, tj. 24 września, strona polska przesłała na ręce J. Masaryka projekt deklaracji o przyszłej konfederacji obu państw. Strona czechosłowacka projekt przyjęła, lecz już w listopadzie 1942 E. Beneš podważył ideę konfederacji. W zamian zaproponowano zawarcie układu sojuszniczego z Polską na 20 lat (formalnie nastąpiło to 20 listopada 1942). question: Co było powodem powrócenia konceptu porozumieniu monachijskiego?', 'output': 'wymiana listów Ripka – Stroński'}


In [7]:
tokenizer = T5Tokenizer.from_pretrained('allegro/plt5-base')

def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding='max_length', truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        outputs = tokenizer(examples['output'], padding='max_length', truncation=True, max_length=128)
    
    inputs['labels'] = outputs['input_ids']
    return inputs
    

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/56618 [00:00<?, ? examples/s]

Map:   0%|          | 0/7060 [00:00<?, ? examples/s]

In [9]:
print(train_data[0])

{'input': 'context: Projekty konfederacji zaczęły się załamywać 5 sierpnia 1942. Ponownie wróciła kwestia monachijska, co uaktywniło się wymianą listów Ripka – Stroński. Natomiast 17 sierpnia 1942 doszło do spotkania E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Polscy dyplomaci zaproponowali podpisanie układu konfederacyjnego. W następnym miesiącu, tj. 24 września, strona polska przesłała na ręce J. Masaryka projekt deklaracji o przyszłej konfederacji obu państw. Strona czechosłowacka projekt przyjęła, lecz już w listopadzie 1942 E. Beneš podważył ideę konfederacji. W zamian zaproponowano zawarcie układu sojuszniczego z Polską na 20 lat (formalnie nastąpiło to 20 listopada 1942). question: Co było powodem powrócenia konceptu porozumieniu monachijskiego?', 'output': 'wymiana listów Ripka – Stroński', 'input_ids': [12634, 22091, 399, 291, 2958, 273, 19605, 6869, 271, 298, 2256, 7465, 394, 540, 2142, 259, 17542, 13760, 10331, 9511, 322, 31220, 261

In [38]:
config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,  r=16, lora_alpha=16, lora_dropout=0.1)
model = T5ForConditionalGeneration.from_pretrained('allegro/plt5-base').to(device)
model = get_peft_model(model, config)

In [39]:
training_args = TrainingArguments(
    output_dir='./results',
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=1000,
    save_total_limit=3,
    save_steps=1000, 
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    weight_decay=0.01
)


In [40]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [41]:
torch.cuda.empty_cache()

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

In [43]:
trainer.train()

Step,Training Loss,Validation Loss
1000,95.692400,41.791786
2000,8.991800,7.595444
3000,6.955500,6.186247
4000,5.755900,4.864702
5000,4.505900,3.662211
6000,3.706600,3.097040
7000,3.557200,2.929034


TrainOutput(global_step=7076, training_loss=23.558995421017677, metrics={'train_runtime': 15051.4084, 'train_samples_per_second': 7.523, 'train_steps_per_second': 0.47, 'total_flos': 8.292752218600243e+16, 'train_loss': 23.558995421017677, 'epoch': 1.999646755448797})

In [44]:
trainer.save_model("./final_model2")

In [8]:
config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,  r=16, lora_alpha=16, lora_dropout=0.1)
model_path = "./final_model2"
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
model = get_peft_model(model, config)
tokenizer = T5Tokenizer.from_pretrained('allegro/plt5-base')

In [9]:
import re

# Funkcja do oczyszczania odpowiedzi modelu z nieoczekiwanych znaków
def clean_text(text):
    pattern = r'[^a-zA-ZąćęłńóśżźĄĆĘŁŃÓŚŻŹ0-9.,!?;:\'"() ]'
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

In [10]:
def get_predictions(data, model, tokenizer, device='cuda'):
    predictions = []
    
    model = model.to(device)
    
    for input_text, output_text in zip(data['input'], data['output']):
        question = input_text.split("question:")[1].strip()
        context = input_text.split("context:")[1].split("question:")[0].strip()

        inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        generated_ids = model.generate(
            input_ids=inputs['input_ids'], 
            attention_mask=inputs['attention_mask'], 
            max_length=128
        )

        prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        prediction = clean_text(prediction)
        predictions.append(prediction)

    return predictions



In [11]:
def extract_question(input_text):
    return input_text.split("question:")[-1].strip()

In [12]:
squad_metric = evaluate.load("squad")

def compute_metrics(predictions, references):

    formatted_references = [{"id": str(i), "answers": {"text": [ref], "answer_start": [0]}} for i, ref in enumerate(references)]
    formatted_predictions = [{"id": str(i), "prediction_text": pred} for i, pred in enumerate(predictions)]
    
    results = squad_metric.compute(predictions=formatted_predictions, references=formatted_references)
    
    return results

In [13]:
predictions = get_predictions(val_data, model, tokenizer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
references = val_data['output']

In [26]:
print(f"Liczba próbek w predictions: {len(predictions)}")
print(f"Liczba próbek w references: {len(references)}")

Liczba próbek w predictions: 7060
Liczba próbek w references: 7060


In [15]:
predictions_text = [item for item in predictions]
references_text = [item for item in references] 

metrics = compute_metrics(predictions_text, references_text)

print(metrics)

{'exact_match': 0.09915014164305949, 'f1': 4.577592185746859}


### Example results on validation dataset

In [16]:
import random 
for i in range(10):
    index = random.randint(0, len(predictions))
    question = extract_question(val_data[index]["input"])
    print(f"Question: {question}")
    prediction = predictions_text[index]
    print(f"Prediction: {prediction}")
    reference = references_text[index]
    print(f"Reference: {reference}")


Question: Jaki bramkarz zastąpił Szczęsnego po jego odejściu z Juventusu?
Prediction:  w lidze włoskiej. Wojciech Szczęsny w Juventusie.
Reference: Gianluigi Donnarumma
Question: Z jaką zawodniczką wygrała Novotna w półfinale?
Prediction:  w finale z Hingis.
Reference: Martiną Hingis
Question: Na jakich aspektach miał skupić się, jako ambasador USA w Polsce, zgodnie z jego przemówieniem z końca 2021 roku?
Prediction:  w Polsce, a nie w Polsce.. Marek Brzeziński, jako ambasador USA w Polsce
Reference: bezstronnym sądownictwie, niezależnych mediach i poszanowaniu praw człowieka dla wszystkich, w tym osób LGBTQI+ i członków innych mniejszości
Question: Jaki był budżet filmu "Wszystko o Stevenie"?
Prediction:  w 2009 roku. "Wszystko o Stevenie":
Reference: 33,8 mln dol
Question: W jakim mieście rozgrywa się akcja wszystkich ekranizacji Moralności pani Dulskiej?
Prediction:  w Krakowie. W jakim mieście rozgrywa się akcja wszystkich ekranizacji?
Reference: Krakowie
Question: Na czym polegał 

### Best results on validation dataset

In [29]:
def compute_metrics_for_examples(predictions, references, val_data):
    formatted_references = [{"id": str(i), "answers": {"text": [ref], "answer_start": [0]}} for i, ref in enumerate(references)]
    formatted_predictions = [{"id": str(i), "prediction_text": pred} for i, pred in enumerate(predictions)]
    
    example_metrics = []
    for i, (pred, ref) in enumerate(zip(formatted_predictions, formatted_references)):
        result = squad_metric.compute(predictions=[pred], references=[ref])
        question = extract_question(val_data[i]["input"])
        
        example_metrics.append({
            "id": pred["id"],
            "question": question,
            "prediction": pred["prediction_text"],
            "reference": ref["answers"]["text"][0],
            "exact_match": result["exact_match"],
            "f1": result["f1"]
        })
    
    return example_metrics

In [50]:
example_metrics = compute_metrics_for_examples(predictions_text, references_text, val_data)

df_metrics = pd.DataFrame(example_metrics)

sorted_by_f1 = df_metrics.sort_values(by="f1", ascending=False)
sorted_by_em = df_metrics.sort_values(by="exact_match", ascending=False)

In [31]:
def display_top_examples(sorted_df, metric, top_n):
    print(f"Top {top_n} examples with highest {metric}:")
    for i, row in sorted_df.head(top_n).iterrows():
        print(f"\nExample {i + 1}:")
        print(f"Question: {row['question']}")
        print(f"Prediction: {row['prediction']}")
        print(f"Reference: {row['reference']}")
        print(f"{metric}: {row[metric]:.2f}")

In [51]:
display_top_examples(sorted_by_f1, 'f1', 10)

Top 10 examples with highest f1:

Example 381:
Question: Gdzie znajdowała się posiadłość Wybickiego?
Prediction:  w Manieczkach.
Reference: w Manieczkach
f1: 100.00

Example 1158:
Question: Gdzie Drygin był klasyfikowany na 73. pozycji w supergigancie?
Prediction:  w Les Orres?
Reference: w Les Orres
f1: 100.00

Example 2110:
Question: W jakiej miejscowości powstał klasztor stworzony z myślą o Rognedzie?
Prediction:  w Izasławiu.
Reference: w Izasławiu
f1: 100.00

Example 1671:
Question: W jakim mieście miał miejsce pierwszy festiwal reggae?
Prediction:  w Montego Bay?
Reference: w Montego Bay
f1: 100.00

Example 4933:
Question: Gdzie przeprowadzono pierwsze testy w locie unowocześnionej wersji tego modelu?
Prediction:  w Kapustin Jarze.
Reference: w Kapustin Jarze
f1: 100.00

Example 3387:
Question: Jakie relikwie zostały przywiezione do Łucka?
Prediction:  św. Recessa Męczennika.
Reference: św. Recessa Męczennika
f1: 100.00

Example 6665:
Question: Jaki utwór był reprezentacyjny dla 

In [52]:
display_top_examples(sorted_by_em, 'exact_match', 10)

Top 10 examples with highest exact_match:

Example 1158:
Question: Gdzie Drygin był klasyfikowany na 73. pozycji w supergigancie?
Prediction:  w Les Orres?
Reference: w Les Orres
exact_match: 100.00

Example 4933:
Question: Gdzie przeprowadzono pierwsze testy w locie unowocześnionej wersji tego modelu?
Prediction:  w Kapustin Jarze.
Reference: w Kapustin Jarze
exact_match: 100.00

Example 2110:
Question: W jakiej miejscowości powstał klasztor stworzony z myślą o Rognedzie?
Prediction:  w Izasławiu.
Reference: w Izasławiu
exact_match: 100.00

Example 6665:
Question: Jaki utwór był reprezentacyjny dla krążka 1991?
Prediction:  Lawa?
Reference: Lawa
exact_match: 100.00

Example 381:
Question: Gdzie znajdowała się posiadłość Wybickiego?
Prediction:  w Manieczkach.
Reference: w Manieczkach
exact_match: 100.00

Example 1671:
Question: W jakim mieście miał miejsce pierwszy festiwal reggae?
Prediction:  w Montego Bay?
Reference: w Montego Bay
exact_match: 100.00

Example 3387:
Question: Jakie 

## Test dataset

In [17]:
from datasets import Dataset

def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

answers = load_jsonl("answers.jl")
passages = load_jsonl("passages.jl")
relevant = load_jsonl("relevant.jl")
questions = load_jsonl("questions.jl")


In [18]:
answers = [a for a in answers if a["score"] == "1"]

passages_dict = {p["_id"]: p["text"] for p in passages}
questions_dict = {q["_id"]: q["text"] for q in questions}

data = []
for rel in relevant:
    passage_text = passages_dict[rel["passage-id"]]
    question_text = questions_dict[rel["question-id"]]
    answer = next((a["answer"] for a in answers if a["question-id"] == rel["question-id"]), None)
    
    if answer:
        data.append({
            "input": f"context: {passage_text} question: {question_text}",
            "output": answer
        })


test_dataset = Dataset.from_dict({"input": [d["input"] for d in data], "output": [d["output"] for d in data]})

print(test_dataset)


Dataset({
    features: ['input', 'output'],
    num_rows: 573
})


In [42]:
test_dataset[0]

{'input': 'context: Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1. question: Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?',
 'output': 'Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.'}

In [19]:
predictions_test = get_predictions(test_dataset, model, tokenizer)

In [20]:
references_test = test_dataset['output']

In [21]:
predictions_test_text = [item for item in predictions_test]
references_test_text = [item for item in references_test] 

metrics_test = compute_metrics(predictions_test_text, references_test_text)

print(metrics_test)

{'exact_match': 0.6980802792321117, 'f1': 9.064270335651074}


### Example results on test dataset

In [22]:
for i in range(10):
    index = random.randint(0, len(predictions_test_text))
    question = extract_question(test_dataset[index]["input"])
    print(f"Question: {question}")
    prediction = predictions_test_text[index]
    print(f"Prediction: {prediction}")
    reference = references_test_text[index]
    print(f"Reference: {reference}")

Question: Co robi ambasador przy organizacji międzynarodowej?
Prediction: a przy organizacji międzynarodowej?"; "art. 20."
Reference: 1) reprezentuje Rzeczpospolitą Polską wobec organizacji, 2) utrzymuje łączność między Rzecząpospolitą Polską a organizacją, 3) prowadzi rokowania z organizacją i w ramach organizacji, 4) zaznajamia się z działalnością prowadzoną przez organizację i przekazuje właściwym organom władzy publicznej Rzeczypospolitej Polskiej informację na temat jej działalności, 5) zapewnia udział Rzeczypospolitej Polskiej w pracach organizacji, 6) chroni interesy Rzeczypospolitej Polskiej, jej obywateli oraz polskich osób prawnych w stosunkach z organizacją, 7) popiera realizację celów i zasad organizacji przez współpracę z organizacją i w ramach organizacji, 8) uczestniczy, poza granicami Rzeczypospolitej Polskiej oraz w zakresie przedmiotu działalności organizacji, w czynnościach przedstawicieli organów władzy publicznej w zakresie prowadzonych przez nich negocjacji i pode

Analizując odpowiedzi modelu, można zauważyc pewne powtarzające się mankamenty. Przede wszystkim odpowiedzi modelu często są niepełne, urwane. Model ma tendencje do umieszczania w odpowiedzi fragmentów, które pojawiły się w pytaniu, a także numerów artykułów oraz powtarzania sekwencji tych samych słów lub symboli. Ponadto model generuje nadmiernie znaki  interpunkcyjne w miejscach, w których nie powinny się one pojawić, np. rozpoczynanie odpowiedzi od przecinka lub kropki. Model generował też nieoczekiwane symbole przypominające znaki z alfabetu chińskiego lub japońskiego, ale zostały one odflitrowane za pomocą wyrażenia regularnego dla zwiększenia czytelności. 

### Best results on test dataset

In [60]:
example_metrics_test = compute_metrics_for_examples(predictions_test_text, references_test_text, test_dataset)

df_metrics_test = pd.DataFrame(example_metrics_test)

sorted_by_f1_test = df_metrics_test.sort_values(by="f1", ascending=False)
sorted_by_em_test = df_metrics_test.sort_values(by="exact_match", ascending=False)

In [61]:
display_top_examples(sorted_by_f1_test, 'f1', 10)

Top 10 examples with highest f1:

Example 354:
Question: Jaki organ wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
Prediction:  Prezes Urzędu Ochrony Konkurencji i Konsumentów.
Reference: Prezes Urzędu Ochrony Konkurencji i Konsumentów.
f1: 100.00

Example 300:
Question: Kiedy została sporządzona Międzynarodowa Konwencja Przeciwko Braniu Zakładników?
Prediction:  w dniu 18 grudnia 1979 r.
Reference: w dniu 18 grudnia 1979 r.
f1: 100.00

Example 358:
Question: Jeśli ustawa nie stanowi inaczej, w jakim postępowaniu dokonuje czynności prokurator?
Prediction:  w postępowaniu przygotowawczym
Reference: W postępowaniu przygotowawczym.
f1: 100.00

Example 326:
Question: Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
Prediction:  Prezes Urzędu Ochrony Konkurencji i Konsumentów.
Reference: Prezes Urzędu Ochrony Konkurencji i Konsumentów
f1: 100.00

Example 192:
Question: Czy w każdej gminie znajduje się wojewódzka biblioteka publiczna?
Prediction:  w każdej

In [62]:
display_top_examples(sorted_by_em_test, 'exact_match', 5)

Top 5 examples with highest exact_match:

Example 326:
Question: Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
Prediction:  Prezes Urzędu Ochrony Konkurencji i Konsumentów.
Reference: Prezes Urzędu Ochrony Konkurencji i Konsumentów
exact_match: 100.00

Example 358:
Question: Jeśli ustawa nie stanowi inaczej, w jakim postępowaniu dokonuje czynności prokurator?
Prediction:  w postępowaniu przygotowawczym
Reference: W postępowaniu przygotowawczym.
exact_match: 100.00

Example 354:
Question: Jaki organ wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
Prediction:  Prezes Urzędu Ochrony Konkurencji i Konsumentów.
Reference: Prezes Urzędu Ochrony Konkurencji i Konsumentów.
exact_match: 100.00

Example 300:
Question: Kiedy została sporządzona Międzynarodowa Konwencja Przeciwko Braniu Zakładników?
Prediction:  w dniu 18 grudnia 1979 r.
Reference: w dniu 18 grudnia 1979 r.
exact_match: 100.00

Example 387:
Question: zamawiający może udzielić zamówienia w tryb

In [23]:
comparison_data = {
    'Metric': ['Exact Match', 'F1'],
    'Validation': [metrics['exact_match'], metrics['f1']],
    'Test': [metrics_test['exact_match'], metrics_test['f1']]
}
comparison_df = pd.DataFrame(comparison_data)

print(comparison_df)

        Metric  Validation     Test
0  Exact Match    0.099150  0.69808
1           F1    4.577592  9.06427


## Questions 

1. Does the performance on the validation dataset reflects the performance on your test set?
   
   Model osiągnął lepsze wyniki na zbiorze testowym niż na zbiorze walidacyjnym, co jest dość zaskakujące, jednak w obu przypadkach wyniki pozostawiały wiele do życzenia. W przypadku zbioru walidacyjnego metryki *F1* i *exact match* wyniosły odpowiednio: 4.6 i 0.1, natomiast w przypadku zbioru testowego *F1* osiągnęło wartość 9.1, a *exact_match* 0.7. Lepsze wyniki modelu na zbiorze testowym mogą wynikać z kilku czynników, związanych z chcarakterystyką obu zbiorów. Zbiór testowy jest bardziej jednorodny - wszystkie pytania dotyczą przepisów prawa, natomiast w zbiorze walidacyjnym mamy pytania z wielu różnych dziedzin. Podobnie w przypadku kontekstów w zbiorze PoQuAD teksty są bardziej zróżnicowane, zarówno pod względem treści, jak i formy, co przekłada się także na odpowiedzi, które również mocno się od siebie rożnią (długością, treścią, formą). Natomiast w przypadku zbioru Simple legal questions wszystkie konteksty są cytatami z przepisów prawa, a więc są bardziej spójne. Podobnie rzecz się ma jeśli chodzi o odpowiedzi. Często ta sama odpowiedź pasuje do wielu pytań. Te czynniki mogą sprawiać, że odpowiadnie na pytania ze zbioru testowego jest dla modelu nieco łatwiejsze. Należy również wziąć pod uwagę znaczące różnice w liczebności obu zbiorów (zbiór walidacyjny - ponad 7 tys. przykładów, zbiór testowy - niecałe 600 przykładów).
2. What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason
   for that?

   Odpowiedzi modelu pozostawiają wiele do życzenia. Model ma tendencję do generowania nieoczekiwanych symboli, przypominających znaki z alfabetu chińskiego lub japońskiego (usunięte za pomocą wyrażenie regularnego dla zwiększenia czytelności) oraz znaków interpunkcyjnych w miejscach, w których nie powinny się one pojawić, np. kropki i przecinki na początku odpowiedzi, odpowiedzi zakończone znakiem zapytania. Poza tym model często generuje odpowiedzi niepełne, urwane lub generuje ciągi tych samych słów lub znaków. Sądzę, że opisane mankamenty wynikają przede wszystkim z ograniczeń związanych z trenowaniem modelu, takich jak ograniczone zasoby sprzętowe, czas, zbiór danych. Ze względu na wspomniane ograniczenia model trenowany był tylko przez 2 epoki. Natomiast zadanie generatywnego odpowiadania na pytania jest zadaniem trudnym, często wymaga zdolności pewnego rodzaju "rozumowania", np. w pytananiach zaczynających się od "czy", które wymagają zrozumienia kontekstu, aby udzielić odpowiedzi "tak" lub "nie", która nie pada bezpośrednio. Trening, który byłam w sptanie przeprowadzić na dostępnuch zasobach, mógł okazać się niewystarczający do tego zadania. 
3. Why extractive question answering is not well suited for inflectional languages?

   Ekstrakcyjne odpowiadanie na pytania nie jest odpowiednie dla języków fleksyjnych ze względu na różnorodność form, w jakich mogą występować wyrazy. Konkretne pytanie determinuje użycie określonych form wyrazów w odpowiedzi, często różnych od tych użytych w kontekście. Proste wycinanie fragmentów kontekstu i umieszczanie ich jako odpowiedzi skutkowałoby tworzeniem wypowiedzi niepoprawnych gramatycznie. 